**PREDICTING CHURN MEANS FINDING IF A CUSTOMER WILL LEAVE OR NOT.**

**THE DATASET HAS 13 FEATURES AMONG WHICH ALL ARE INDEPENDENT EXCEPT THE LAST ONE WHICH IS DEPENDENT. ANN IS APPLIED TO CREATE A MODEL TO PREDICT THE CHURN.**

**I CREATED A ANN WITH 2 HIDDEN LAYERS BOTH WITH 6 NEURONS.**

In [ ]:
#IMPORTING LIBRARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
#IMPORTING DATASET
ds = pd.read_csv('../input/churn-modelling/Churn_Modelling.csv')
ds.head()

In [ ]:
#REMOVING UNNECESSARY FEATURES
ds.drop(['RowNumber', 'CustomerId', 'Surname'],axis = 1,inplace = True)

In [ ]:
#DATASET NOW HAS 10000 ROWS AND 11 COLUMNS
ds.shape

In [ ]:
#CHECKING FOR NULL VALUES
ds.isnull().sum()

**NO MISSING VAUES**

In [ ]:
#FEATURES 1 AND 2 ARE CATEGORICAL THEREFORE WE HAVE TO CONVERT THAT CATEGORICAL DATA
ds.info()

In [ ]:
ds.columns

In [ ]:
plt.figure(figsize=(18,10))
col = ['Geography','Gender','Tenure','NumOfProducts', 'HasCrCard','IsActiveMember']
c = 1
for i in col:
    if c < 7:
        plt.subplot(2,3,c)
        sns.countplot(x = i,data =ds,hue = 'Exited')
        plt.xlabel(i)
    c += 1
    plt.tight_layout()

**-MOST CHURNED CUSTOMERS ARE FROM GERMNAY.**

**-NUMBER OF FEMALE CUSTOMERS CHURNED ARE MORE AS COMPARED TO MALE CUSTOMERS.**

**-TENURE BETWEEN 1 - 9 IS HIGH.**

**-MOST CUSTOMERS CHURNED HAS 1 PRODUCT.**

**-PEOPLE HAVING HASH CARD ARE MORE LIKELY TO CHURN.**

**-WHEN PEOPLE BECOME INACTIVE THEY ARE MOST LIKELY TO EXIT THE BANK.**


In [ ]:
sns.countplot(x = 'Exited', data = ds)

**AROUND 2000  CUSTOMERS HAVE CHURNED**

In [ ]:
plt.figure(figsize=(12,10))
col = ['CreditScore','Age','Balance','EstimatedSalary']
c = 1
for i in col:
    if c < 5:
        plt.subplot(2,2,c)
        sns.violinplot(x = 'Exited',y = i,data =ds )
        plt.xlabel(i)
    c += 1
    plt.tight_layout()

**-CHURNED CUSTOMERS HAD AN AVERAGE CREDIT SCORE AROUND 650, BUT CREDIT SCORE DOESN'T HAVE ANY AFFECT ON CHURNING.**

**-CHURNED CUSTOMERS HAD AN AVERAGE AGE OF 45,AGED PEOPLE WERE FOUND TO BE LEAVING MORE.**

**-CHURNED CUSTOMERS HAD AN AVERAGE BALANCE BETWEEN 100000 - 150000.**

**-SALARY DOESN'T HAVE ANT AFFECT ON CHURNING**

In [ ]:
plt.figure(figsize = (12,10))
sns.heatmap(ds.corr(),annot = True)

**NO SIGNIFICANT CORRELATION**

In [ ]:
#DIVIDING DATASET IN DEPENDENT AND INDEPENDENT VARIABLE
x = ds.iloc[:,:-1].values
y = ds.iloc[:,-1].values

In [ ]:
#APPLYING LABEL ENCODING ON FEATURE 'GENDER'
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])

In [ ]:
#APPLYING ONE HOT ENCODING ON FEATURE 'GEOGRAPHY'
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [ ]:
#SPLITTING DATASET IN TRAINING AND TESTING SET
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 0)

In [ ]:
#FEATURE SCALING IS NECESSARY IN ANN
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
#CREATING THE ANN AS SEQUENCE OF LAYERS
ann = tf.keras.Sequential()

In [ ]:
#ADDING FIRST HIDDEN LAYER WITH 6 NEURONS, THE INPUT LAYER WILL BE ADDED AUTOMATICALLY,
ann.add(tf.keras.layers.Dense(units = 6,activation = 'relu'))

In [ ]:
#ADDING 2ND HIDDEN LAYER WITH 6 NEURONS
ann.add(tf.keras.layers.Dense(units = 6,activation = 'relu'))

In [ ]:
#ADDING OUTPUT LAYER WITH 1 NEURON , AS THIS IS A BINARY CLASSIFICATION
ann.add(tf.keras.layers.Dense(units = 1,activation = 'sigmoid'))

In [ ]:
#COMPILING THE ANN USING STOCHASTIC GRADIENT DESCENT (optimizer = 'adam')
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
#TRAINING THE ANN WITH BATCH SIZE OF 32 (THIS IS A BATCH LEARNING)
ann.fit(x_train, y_train,batch_size = 32, epochs = 100)

In [ ]:
#GETTING ACCURACY AND CONFUSION MATRIX
from sklearn.metrics import confusion_matrix,accuracy_score
y_pred = ann.predict(x_test)
y_pred  = y_pred > 0.5
cm = confusion_matrix(y_pred,y_test)
ac = accuracy_score(y_pred,y_test)

In [ ]:
sns.heatmap(cm,annot = True)
plt.title('CONFUSION MATRIX')
print('Accuracy - {}%'.format(ac*100))